In [1]:
from IPython.display import HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from pandas.tseries.offsets import DateOffset

import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from copy import deepcopy
import datetime
import calendar
from typing import List

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from lightgbm import Dataset


In [5]:
import sys
sys.path.append('../..')
from utils.analyze import compute_metrics
from utils.metrics import Bias, Wape, Count, Sum, Mape, MSE

### Data Loader

In [6]:
data = pd.read_csv('data/train.csv')
data['dt'] = pd.to_datetime(data['dt'])

In [7]:
# functions

def create_train_df(df: pd.DataFrame, 
                    START_PREDICT_DATE: str,
                    END_PREDICT_DATE: str
                   ):
    
    START_TRAIN_DATE = pd.to_datetime('2016-01-02')
    START_PREDICT_DATE = pd.to_datetime(START_PREDICT_DATE)
    END_PREDICT_DATE = pd.to_datetime(END_PREDICT_DATE)
    
    range_date = pd.date_range(START_TRAIN_DATE, END_PREDICT_DATE, freq='D')
    unique_shop_id = df['id'].unique().tolist()
    
    train_grid = pd.DataFrame(list(product(unique_shop_id,
                                           range_date)),
                              columns = ['id', 'dt']
                             )
    df_train = train_grid.merge(df, how = 'left', on = ['id', 'dt'])
    df_train['sales'] = df_train['target'].copy()
    df_train['target'] = df_train['target'].fillna(0)
    df_train['split'] = np.where(df_train['dt'] >= START_PREDICT_DATE, 'test', 'train')
    df_train.loc[df_train['dt'] >= START_PREDICT_DATE, 'target'] = np.nan 

    return df_train

def create_calendar_vars(df, date_col):
    """
    Create calendar vars
    """

    def week_of_month(dt):
        first_day = dt.replace(day=1)
        dom = dt.day
        adjusted_dom = dom + first_day.weekday()
        return int(np.ceil(adjusted_dom / 7.0))

    df_ = df.copy()
    df_['year'] = df_[date_col].dt.year.astype('int')
    df_['quarter'] = df_[date_col].dt.quarter.astype('int')
    df_['month'] = df_[date_col].dt.month.astype('int')
    df_['dayofweek'] = df_[date_col].dt.dayofweek.astype('int')
    df_['dayofyear'] = df_[date_col].dt.dayofyear.astype('int')
    df_['dayofmonth'] = df_[date_col].dt.day.astype('int')
    df_['week'] = df_[date_col].dt.isocalendar().week.astype('int') 
    df_['week'] = df_['week'].astype('int')
    df_['is_weekend'] = pd.Series(df_[date_col]).apply(lambda x : x.weekday() in [5, 6]).values 
    df_['is_weekend'] = df_['is_weekend'].astype('category')
    df_['week_of_month'] = pd.Series(df_[date_col]).apply(week_of_month).values
    df_['week_of_month'] = df_['week_of_month'].astype('int')
    df_['first_day_month'] = df_['dayofmonth'].apply(lambda x: x == 1).astype('category')
    df_['second_day_month'] = df_['dayofmonth'].apply(lambda x: x == 2).astype('category')
    df_['is_evenweek'] = df_['week'] // 2 == 0 
    df_['is_evenweek'] = df_['is_evenweek'].astype('category')
    df_['week_of_month + week'] = df_['week_of_month'].astype('str') + '-'  + df_['week'].astype('str')
    df_['week_of_month + week'] = df_['week_of_month + week'].astype('category')
    df_['week + quarter'] = df_['week'].astype('str') + '-'  + df_['quarter'].astype('str') 
    df_['week + quarter'] = df_['week + quarter'].astype('category')
    df_['dayofmonth + quarter'] = df_['dayofmonth'].astype('str') + '-'  + df_['quarter'].astype('str')
    df_['dayofmonth + quarter'] = df_['dayofmonth + quarter'].astype('category')
    df_['is_evenweek + dayofweek'] = df_['is_evenweek'].astype('str') + '-'  + df_['dayofweek'].astype('str')
    df_['is_evenweek + dayofweek'] = df_['is_evenweek + dayofweek'].astype('category')
    return df_



def get_start_end_dates(df, n_splits, col_dates='dt'):
    """
    create dates
    """
    max_date = df[col_dates].max()
    
    from_date = max_date - DateOffset(months=n_splits-1)     
    beg_date = datetime.datetime(from_date.year, from_date.month, 1)
    for i in range(n_splits):
        n_days_in_that_month = calendar.monthrange(beg_date.year, beg_date.month)[1]
        end_date = datetime.datetime(beg_date.year, beg_date.month, n_days_in_that_month)
        yield (beg_date, end_date)
    
        beg_date = end_date + datetime.timedelta(days=1)

In [8]:
def create_lags(df, group_columns: List[str], target_column: str, lags_range: list):
    """
    Create lags
    """
    df_ = df.copy()
    for lag in lags_range:
        df_[f'Lags_{lag}'] = df_.groupby(group_columns)[target_column].shift(lag)
        
    return df_


def create_group_means(df, target_column: str):
    """
    Create grouped means
    """
    df_ = df.copy()
    df_['mean_id'] = df_.groupby(['id'])[target_column].transform(lambda x: x.mean())
    df_['mean_year'] = df_.groupby(['year'])[target_column].transform(lambda x: x.mean())
    df_['mean_quarter'] = df_.groupby(['quarter'])[target_column].transform(lambda x: x.mean())
    df_['mean_week'] = df_.groupby(['week'])[target_column].transform(lambda x: x.mean())
    df_['mean_dayofweek'] = df_.groupby(['dayofweek'])[target_column].transform(lambda x: x.mean())
    df_['mean_dayofweek_even_week'] = df_.groupby(['dayofweek', 'is_evenweek'])[target_column].transform(lambda x: x.mean())
    
    df_['mean_dayofweek_even_id'] = df_.groupby(['dayofweek', 'is_evenweek', 'id'])['target'].transform(lambda x: x.mean())
    df_['max_dayofweek_even_id'] = df_.groupby(['dayofweek', 'is_evenweek', 'id'])['target'].transform(lambda x: x.max())
    df_['min_dayofweek_even_id'] = df_.groupby(['dayofweek', 'is_evenweek', 'id'])['target'].transform(lambda x: x.min())
    df_['diff_dayofweek_even_id'] = df_.groupby(['dayofweek', 'is_evenweek', 'id'])['target'].transform(lambda x: x.max() - x.min())
    
    df_['mean_dayofweek_week_of_month_id'] = df_.groupby(['dayofweek', 'week_of_month', 'id'])['target'].transform(lambda x: x.mean())
    df_['max_dayofweek_week_of_month_id'] = df_.groupby(['dayofweek', 'week_of_month', 'id'])['target'].transform(lambda x: x.max())
    df_['min_dayofweek_week_of_month_id'] = df_.groupby(['dayofweek', 'week_of_month', 'id'])['target'].transform(lambda x: x.min())
    df_['diff_dayofweek_week_of_month_id'] = df_.groupby(['dayofweek', 'week_of_month', 'id'])['target'].transform(lambda x: x.max() - x.min())

                                                   
    return df_


def create_rollings_means(df, target_column: str):
    """
    Create rolling means
    """
    
    df_ = df.copy()
    df_['rolling_mean_id'] = df_.groupby(['id'])[target_column].transform(
        lambda s: s.shift(HORIZON).rolling(window=7, min_periods=1, center=False).agg('mean'))
    df_['rolling_mean_id_28'] = df_.groupby(['id'])[target_column].transform(
        lambda s: s.shift(HORIZON).rolling(window=28, min_periods=1, center=False).agg('mean'))
        
    return df_


def create_seasonal_vars(df: pd.DataFrame, group_columns: List[str], target_column=str):
    """
    Create seasonal features
    """
    train = df[df['split'] == 'train']
    train['trend'] = train.groupby(group_columns)[target_column].transform(
        lambda s: s.shift(HORIZON).rolling(window=500, min_periods=30, center=True).agg('mean')
    )
    train['trend'] = train['trend'].fillna(method='bfill')
    train["detrended"] = train[target_column] - train["trend"]
    
    coef = train.groupby(group_columns, as_index = False)[['detrended']].mean()
    coef = coef.rename(columns = {'detrended': 'seasonal'})
    
    # Клипануть сезонные коэффициенты
    
    return coef



In [9]:
def filter_dataset(df):
    mask_na = (df['dt'] >= pd.to_datetime('2016-07-01'))\
                & (df['dt'] < pd.to_datetime('2017-01-01'))\
                & (df['id'].isin([4, 8, 10, 13, 17])
                  )
    early_mask = (df['dt'] <= pd.to_datetime('2016-03-01')) 
    mask = mask_na + early_mask
    
    return df[~mask]

In [10]:
def get_valid_index(train: 
                    pd.DataFrame, 
                    valid_size: int = 0.1,
                    random_seed = 0
                   ):
    """
    get valid index and drop them form train
    """
    np.random.seed(random_seed)
    valid_size = np.ceil(len(train) * valid_size).astype(int)
    valid_idx = np.random.choice(train.index, valid_size, replace = False)
    valid = train.loc[valid_idx]
    
    train.drop(valid_idx, axis = 0, inplace = True)
    
    return valid
    

In [11]:
def prepare_datesets(df, features, valid_size, random_seed):
    """
    Prepare dataset for lgbm
    
    """
    
    train = df[(df['split'] == 'train')]
    test = df[df['split'] == 'test']
    valid = get_valid_index(train,
                           valid_size,
                           random_seed,
                       )
    
    cat_columns = train[features].select_dtypes(include='category').columns.tolist()
    
    # lgbm datasets
    train_ds = lgb.Dataset(train[features],
                       train['target'],
                       categorical_feature=cat_columns,
                       )
    
    val_ds = lgb.Dataset(valid[features],
                         valid['target'],
                         categorical_feature=cat_columns,
                         reference=train_ds
                        )
    
    return train_ds, val_ds



In [12]:
def get_predictions(df, model):
    df['prediction'] = model.predict(df[features2train])
    df['prediction'] = np.clip(df['prediction'], a_min=0, a_max = None)
    
    # post_processsing
    claster = [ 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 16, 17, 18, 19]
    claster_holidays = [pd.to_datetime('2017-06-04')]
    all_holidays = [pd.to_datetime('2017-05-01'),
                    pd.to_datetime('2017-05-14'),
                    pd.to_datetime('2017-05-25'),
                    pd.to_datetime('2017-04-03'),
                    pd.to_datetime('2017-04-06'),
                   ]

    for shop in claster:
        df.loc[(df['id'] == shop)
              & (df['dt'].isin(claster_holidays)), 
               'prediction'
              ] = 0

    df.loc[(df['dt'].isin(all_holidays)), 
           'prediction'
          ] = 0    
        
    return df

In [13]:
def get_metrics(df):
    metrics = {}

    shops_list = df['id'].unique().tolist()
    for shop in shops_list:
        shop_df = df[df['id'] == shop] 
        y_test = shop_df[shop_df['split'] == 'test']['sales'].fillna(0).copy()
        y_pred = shop_df[shop_df['split'] == 'test']['prediction'].copy()
        y_train = shop_df[shop_df['split'] == 'train']['sales'].fillna(0).copy()
        metric =  mean_absolute_scaled_error(y_test, y_pred, y_train = y_train, sp = HORIZON)
        metrics[shop] = metric
        
    return metrics

        
def plot(df: pd.DataFrame, metrics: dict):
    """
    Draw plots with sales & prediction
    """

    df_agg = df.groupby('id')
    fig, axes = plt.subplots(5, 4, figsize = (20, 20))
    axes = axes.ravel()

    for name, group in df_agg:
        group = group[group['split'] == 'test']
        axes[name].plot(group['dt'], group['sales'], label = 'sales',  color = 'red')
        axes[name].plot(group['dt'], group['prediction'], label = 'prediction',  color = 'black')

        metric = metrics[name]
        axes[name].set_title(f'Sales and Prediction\n for Store № {name} -- {metric}')
        axes[name].legend()
        axes[name].grid()
        axes[name].xaxis.set_major_locator(md.WeekdayLocator(MO))
        axes[name].xaxis.set_tick_params(rotation=90)
    plt.tight_layout()
    

### Перекретсная проверка

In [14]:
data = pd.read_csv('data/train.csv')
data['dt'] = pd.to_datetime(data['dt'])

In [15]:
# define features for training
features2train = ['id', 
                  'year', 'quarter', 'month',
                  'dayofmonth', 
                  'dayofweek', 
                  'week', 'is_weekend',
                  'first_day_month', 'second_day_month',
                  'week_of_month + week',
                  'mean_id',
                  'mean_year',
                  'mean_quarter',
                  'mean_week', 
                  'mean_dayofweek',
                  'mean_dayofweek_even_week',
                  'rolling_mean_id',
                  'rolling_mean_id_28',
                  'mean_dayofweek_even_id',
                  'max_dayofweek_even_id', 
                  'min_dayofweek_even_id',
                  'diff_dayofweek_even_id',     
                  'mean_dayofweek_week_of_month_id',
                  'max_dayofweek_week_of_month_id',
                  'min_dayofweek_week_of_month_id',
                  'diff_dayofweek_week_of_month_id',
                  'Lags_91', 
                  'Lags_29', 
                  'Lags_62', 
                  'Lags_122',
                  'seasonal',
                 ]

# boosting hyperparameters
params = {
    'objective': 'mae', 
    'eta': 0.01, # 0.1
    'max_depth' : -1,
    'seed' : 42, 
    'verbose' : -1,
     'num_threads': 8,
    
}


# Best value (rmse): 0.17825
# 	Best params:
# 		n_estimators: 2000
# 		learning_rate: 0.0032824665592938616
            

# Constants
VALID_SIZE = 0.2
VISUALIZE = False

In [16]:
dates = (get_start_end_dates(data, n_splits=4, col_dates='dt'))

for date in dates:
    # define prediction date
    test_start_date = date[0]
    test_end_date = date[1]
    HORIZON = calendar.monthrange(test_start_date.year, test_start_date.month)[1]
    print(test_start_date, test_end_date, HORIZON)
    
    # create dataframe
    df = create_train_df(data, 
                         test_start_date,
                         test_end_date
                        )
    
    # create features
    df = create_calendar_vars(df, date_col='dt')
    
    # create lags
    
    df = create_lags(df, group_columns=['id'], target_column='target', 
                     lags_range = [28, 29, 30, 31, 62, 91, 122, 365])
    
    # gropued means
    df = create_group_means(df, target_column='target')
    
    # rolling means
    df = create_rollings_means(df, target_column='target')
    
    # seasonal coef
    seasona_coef = create_seasonal_vars(df, 
                                        group_columns=['dayofweek', 'is_evenweek', 'id'], 
                                        target_column='target')
    df = df.merge(seasona_coef, on = ['dayofweek', 'is_evenweek', 'id'], how = 'left')
    
    # filter dataset
    df = filter_dataset(df)
    
    # prepeare lgbm dataset
    train_ds, val_ds = prepare_datesets(df, features2train, valid_size=VALID_SIZE, random_seed=0)
    
    # train
    model = lgb.train(params,
                      train_ds,
                      num_boost_round = 1000,
                      valid_sets=[train_ds, val_ds],
                      verbose_eval=500,
                      evals_result=None,
                      )
    
    # prediction
    df = get_predictions(df, model)
    
    # get metrics
    metrics = get_metrics(df)
    print(f'Mean Smape {np.array(list(metrics.values())).mean()}')
    
    if VISUALIZE:
        plot(df, metrics)
    

2017-03-01 00:00:00 2017-03-31 00:00:00 31
[500]	training's l1: 48.9999	valid_1's l1: 53.4864
[1000]	training's l1: 37.9841	valid_1's l1: 48.3325
Mean Smape 0.15609563912108065
2017-04-01 00:00:00 2017-04-30 00:00:00 30
[500]	training's l1: 47.3794	valid_1's l1: 57.3907
[1000]	training's l1: 37.8847	valid_1's l1: 50.9831
Mean Smape 0.2429270911980609
2017-05-01 00:00:00 2017-05-31 00:00:00 31
[500]	training's l1: 50.9441	valid_1's l1: 59.6672
[1000]	training's l1: 41.7753	valid_1's l1: 54.7217
Mean Smape 0.19389037377165524
2017-06-01 00:00:00 2017-06-30 00:00:00 30
[500]	training's l1: 53.4316	valid_1's l1: 59.2664
[1000]	training's l1: 42.174	valid_1's l1: 53.0589
Mean Smape 0.15045403374672306


#### Baseline
Mean Smape 0.15609563912108065 \
Mean Smape 0.2429270911980609 \
Mean Smape 0.19389037377165524 \
Mean Smape 0.15045403374672306 \

Mean Smape 0.1585581324852377 \
Mean Smape 0.22812083356894738 \
Mean Smape 0.19650267164915525 \
Mean Smape 0.15459359524156494

### Подбор гиперпараметров модели

In [40]:
import optuna

In [61]:
def objective(trial, df, n_splits = 4):
    
    all_metric = []
    dates = (get_start_end_dates(data, n_splits=n_splits, col_dates='dt'))

    for date in dates:
        # define prediction date
        test_start_date = date[0]
        test_end_date = date[1]
        HORIZON = calendar.monthrange(test_start_date.year, test_start_date.month)[1]
        print(test_start_date, test_end_date, HORIZON)

        # create dataframe
        df = create_train_df(data, 
                             test_start_date,
                             test_end_date
                            )

        # create features
        df = create_calendar_vars(df, date_col='dt')

        # create lags

        df = create_lags(df, group_columns=['id'], target_column='target', 
                         lags_range = [28, 29, 30, 31, 62, 91, 122, 365])

        # gropued means
        df = create_group_means(df, target_column='target')

        # rolling means
        df = create_rollings_means(df, target_column='target')

        # seasonal coef
        seasona_coef = create_seasonal_vars(df, 
                                            group_columns=['dayofweek', 'is_evenweek', 'id'], 
                                            target_column='target')
        df = df.merge(seasona_coef, on = ['dayofweek', 'is_evenweek', 'id'], how = 'left')

        # filter dataset
        df = filter_dataset(df)

        # prepeare lgbm dataset
        train_ds, val_ds = prepare_datesets(df, features2train, valid_size=VALID_SIZE, random_seed=0)
        
        params = {
            'objective': 'mae', 
            'seed' : 42, 
            'verbose' : -1,
            'num_threads': 8,
            'early_stopping_rounds':100,
            "n_estimators": trial.suggest_categorical("n_estimators", [1000, 2000, 3000, 5000, 7000, 10000]), # n_estimators: 2000
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.15) # learning_rate: 0.059863266105050765
            #"num_leaves": trial.suggest_int("num_leaves", 12, 120, step=4), # 16
            #"max_depth": trial.suggest_int("max_depth", 1, 6), # 5
            #"lambda_l1": trial.suggest_int("lambda_l1", 0, 10, step=1), # 0
            #"lambda_l2": trial.suggest_int("lambda_l2", 0, 10, step=1), # 8
            #"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 0.95, step=0.1), # 0.9
            #"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 0.95, step=0.1),# 0.6
        }
            

        # train
        model = lgb.train(params,
                          train_ds,
                          num_boost_round = 1000,
                          valid_sets=[train_ds, val_ds],
                          verbose_eval=500,
                          evals_result=None,
                          )

        # prediction
        df = get_predictions(df, model)

        # get metrics
        metrics = get_metrics(df)
        all_metric.append(np.array(list(metrics.values())).mean())
    
    return np.median(all_metric)
    

In [62]:
study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial, df, n_splits = 4)
study.optimize(func, n_trials=50)

[I 2023-08-07 12:18:45,904] A new study created in memory with name: LGBM Regressor


2017-03-01 00:00:00 2017-03-31 00:00:00 31
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 25.0896	valid_1's l1: 44.6969
[1000]	training's l1: 20.9504	valid_1's l1: 43.6188
[1500]	training's l1: 18.9679	valid_1's l1: 43.2919
[2000]	training's l1: 17.9426	valid_1's l1: 43.125
[2500]	training's l1: 17.2846	valid_1's l1: 43.0066
Early stopping, best iteration is:
[2616]	training's l1: 17.1506	valid_1's l1: 42.9941
2017-04-01 00:00:00 2017-04-30 00:00:00 30
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 24.8416	valid_1's l1: 46.6612
[1000]	training's l1: 20.5382	valid_1's l1: 45.2732
[1500]	training's l1: 18.5066	valid_1's l1: 44.5652
[2000]	training's l1: 17.3674	valid_1's l1: 44.2009
[2500]	training's l1: 16.6166	valid_1's l1: 43.9913
[3000]	training's l1: 16.084	valid_1's l1: 43.8837
[3500]	training's l1: 15.6342	valid_1's l1: 43.768
[4000]	training's l1: 15.3213	valid_1's l1: 43.7091
[4500]	training's l1: 15.0737	va

[I 2023-08-07 12:19:18,080] Trial 0 finished with value: 0.20304118040355867 and parameters: {'n_estimators': 5000, 'learning_rate': 0.07988692918026569}. Best is trial 0 with value: 0.20304118040355867.


2017-03-01 00:00:00 2017-03-31 00:00:00 31
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 30.0986	valid_1's l1: 45.9897
[1000]	training's l1: 24.0068	valid_1's l1: 43.5969
[1500]	training's l1: 21.4476	valid_1's l1: 43.006
[2000]	training's l1: 20.0014	valid_1's l1: 42.723
[2500]	training's l1: 18.943	valid_1's l1: 42.5057
[3000]	training's l1: 18.2017	valid_1's l1: 42.342
Early stopping, best iteration is:
[3104]	training's l1: 18.0906	valid_1's l1: 42.3181
2017-04-01 00:00:00 2017-04-30 00:00:00 30
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 28.1913	valid_1's l1: 46.2039
[1000]	training's l1: 23.4251	valid_1's l1: 44.7987
[1500]	training's l1: 21.3215	valid_1's l1: 44.1688
[2000]	training's l1: 20.1653	valid_1's l1: 43.8706
[2500]	training's l1: 19.2486	valid_1's l1: 43.5878
[3000]	training's l1: 18.4501	valid_1's l1: 43.3115
[3500]	training's l1: 17.8128	valid_1's l1: 43.0059
Early stopping, best iteration is

[I 2023-08-07 12:19:49,399] Trial 1 finished with value: 0.20264348374321506 and parameters: {'n_estimators': 5000, 'learning_rate': 0.047897904114985776}. Best is trial 1 with value: 0.20264348374321506.


...

[I 2023-08-07 12:41:48,755] Trial 47 finished with value: 0.178483730195554 and parameters: {'n_estimators': 1000, 'learning_rate': 0.03095189838366851}. Best is trial 37 with value: 0.17040482646838673.


2017-03-01 00:00:00 2017-03-31 00:00:00 31
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 37.8258	valid_1's l1: 48.4933
[1000]	training's l1: 29.967	valid_1's l1: 45.3336
[1500]	training's l1: 25.9984	valid_1's l1: 43.7298
[2000]	training's l1: 24.0565	valid_1's l1: 43.3586
[2500]	training's l1: 22.7345	valid_1's l1: 43.111
[3000]	training's l1: 21.7921	valid_1's l1: 42.9149
[3500]	training's l1: 21.0497	valid_1's l1: 42.7653
[4000]	training's l1: 20.4565	valid_1's l1: 42.6293
[4500]	training's l1: 19.9289	valid_1's l1: 42.5094
[5000]	training's l1: 19.4781	valid_1's l1: 42.4385
Early stopping, best iteration is:
[4939]	training's l1: 19.5291	valid_1's l1: 42.4279
2017-04-01 00:00:00 2017-04-30 00:00:00 30
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 36.4715	valid_1's l1: 49.6315
[1000]	training's l1: 29.2353	valid_1's l1: 46.4842
[1500]	training's l1: 26.1295	valid_1's l1: 45.6162
[2000]	training's l1: 24.0397	v

[I 2023-08-07 12:43:04,103] Trial 48 finished with value: 0.20030489208917562 and parameters: {'n_estimators': 7000, 'learning_rate': 0.021662304261280282}. Best is trial 37 with value: 0.17040482646838673.


2017-03-01 00:00:00 2017-03-31 00:00:00 31
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 59.5938	valid_1's l1: 61.3687
[1000]	training's l1: 42.16	valid_1's l1: 50.3657
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 42.16	valid_1's l1: 50.3657
2017-04-01 00:00:00 2017-04-30 00:00:00 30
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 56.7996	valid_1's l1: 65.7477
[1000]	training's l1: 41.0993	valid_1's l1: 52.3804
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 41.0993	valid_1's l1: 52.3804
2017-05-01 00:00:00 2017-05-31 00:00:00 31
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 60.0851	valid_1's l1: 65.9667
[1000]	training's l1: 46.1936	valid_1's l1: 57.0114
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 46.1936	valid_1's l1: 57.0114
2017-06-01 00:00:00 2017-06-30 00:00:00 30
Training until validation scores don'

[I 2023-08-07 12:43:23,072] Trial 49 finished with value: 0.17298180854435263 and parameters: {'n_estimators': 1000, 'learning_rate': 0.007067306848916712}. Best is trial 37 with value: 0.17040482646838673.


In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")
    